* Spark SQL

In [13]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()
# spark.stop()

* Creating dataframes form csv files

In [14]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.load("persons.csv",
                        format="csv",
                        header=True,
                        inferSchema=True)
df.show()

+-------+----+
|   name| age|
+-------+----+
|   Andy|  30|
|Michael|null|
| Justin|  19|
|Michael|null|
+-------+----+



In [20]:
df.first()

Row(name='Andy', age=30)

* Creating DataFrames from RDD or python lists

In [3]:
spark2 = SparkSession.builder.getOrCreate()
profilesList = [(19, "Justin"), (30, "Andy"),(None, "Michael")]
df = spark2.createDataFrame(profilesList,["age","name"])
df.show()

+----+-------+
| age|   name|
+----+-------+
|  19| Justin|
|  30|   Andy|
|null|Michael|
+----+-------+



* From dataframe to RDD

In [4]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.load( "persons.csv",
                        format="csv",
                        header=True,
                        inferSchema=True)
rddRows = df.rdd
print(rddRows.collect())
rddNames = rddRows.map(lambda row: row.name)
print(rddNames.collect())

[Row(name='Andy', age=30), Row(name='Michael', age=None), Row(name='Justin', age=19), Row(name='Michael', age=None)]
['Andy', 'Michael', 'Justin', 'Michael']


* Operations on dataframe

In [5]:
df.show()
df.show(2)
df.printSchema()
df.count()

+-------+----+
|   name| age|
+-------+----+
|   Andy|  30|
|Michael|null|
| Justin|  19|
|Michael|null|
+-------+----+

+-------+----+
|   name| age|
+-------+----+
|   Andy|  30|
|Michael|null|
+-------+----+
only showing top 2 rows

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



4

In [6]:
df.distinct().show()
df.select("name").show()
df.selectExpr("name","age+1 AS new_age").show()

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
| Justin|  19|
|   Andy|  30|
+-------+----+

+-------+
|   name|
+-------+
|   Andy|
|Michael|
| Justin|
|Michael|
+-------+

+-------+-------+
|   name|new_age|
+-------+-------+
|   Andy|     31|
|Michael|   null|
| Justin|     20|
|Michael|   null|
+-------+-------+



In [7]:
df.filter("age>=25 and age<=40").show()
df.where("age>=25 and age<=40").show()

+----+---+
|name|age|
+----+---+
|Andy| 30|
+----+---+

+----+---+
|name|age|
+----+---+
|Andy| 30|
+----+---+



In [8]:
spark4 = SparkSession.builder.getOrCreate()
sport_to_match = [("ski", "Justin"), ("basketball", "Andy"),("swimming", "Michael")]
df4 = spark4.createDataFrame(sport_to_match,["sport","name"])
dfJoin= df.join(df4, df.name==df4.name)
dfJoin.show()

+-------+----+----------+-------+
|   name| age|     sport|   name|
+-------+----+----------+-------+
| Justin|  19|       ski| Justin|
|   Andy|  30|basketball|   Andy|
|Michael|null|  swimming|Michael|
|Michael|null|  swimming|Michael|
+-------+----+----------+-------+



In [9]:
avg_age = df.agg({"age":"avg"})
avg_age.show()

+--------+
|avg(age)|
+--------+
|    24.5|
+--------+



In [10]:
groupped = df.groupBy("name").avg("age")
groupped.show()

+-------+--------+
|   name|avg(age)|
+-------+--------+
|Michael|    null|
|   Andy|    30.0|
| Justin|    19.0|
+-------+--------+



In [12]:
df.groupBy("name").agg({"age":"avg","name":"count"}).show()

+-------+-----------+--------+
|   name|count(name)|avg(age)|
+-------+-----------+--------+
|Michael|          2|    null|
|   Andy|          1|    30.0|
| Justin|          1|    19.0|
+-------+-----------+--------+



In [36]:
df.sort("name").show()
df.sort("name",ascending=False).show()

+-------+----+
|   name| age|
+-------+----+
|   Andy|  30|
| Justin|  19|
|Michael|null|
|Michael|null|
+-------+----+

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|Michael|null|
| Justin|  19|
|   Andy|  30|
+-------+----+



* Dataframe and SQL 

In [3]:
spark3 = SparkSession.builder.getOrCreate()
profilesList3 = [(19, "Justin"), (30, "Andy"),(None, "Michael"),(40, "Justin")]
df3 = spark3.createDataFrame(profilesList3,["age","name"])
df3.show()

df3.createOrReplaceTempView("people")
selectedPersons=spark3.sql("SELECT * FROM people WHERE age>20 and age<40")
selectedPersons.show()

+----+-------+
| age|   name|
+----+-------+
|  19| Justin|
|  30|   Andy|
|null|Michael|
|  40| Justin|
+----+-------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [4]:
# Operation on two data frame
profilesList4 = [("swimming", "Justin"), ("jogging", "Andy"),("running", "Michael")]
df4  =  spark3.createDataFrame(profilesList4,["sport","name"])
df4.show()
df4.createOrReplaceTempView("LikeSport")

dfPersonLikes=spark3.sql("SELECT * FROM people,LikeSport where people.name=LikeSport.name")
dfPersonLikes.show()

+--------+-------+
|   sport|   name|
+--------+-------+
|swimming| Justin|
| jogging|   Andy|
| running|Michael|
+--------+-------+

+----+-------+--------+-------+
| age|   name|   sport|   name|
+----+-------+--------+-------+
|null|Michael| running|Michael|
|  30|   Andy| jogging|   Andy|
|  19| Justin|swimming| Justin|
|  40| Justin|swimming| Justin|
+----+-------+--------+-------+



In [5]:
nameAvgAgeCount=spark3.sql("SELECT name,avg(age),count(name) FROM people GROUP BY name")
nameAvgAgeCount.show()

+-------+--------+-----------+
|   name|avg(age)|count(name)|
+-------+--------+-----------+
|Michael|    null|          1|
|   Andy|    30.0|          1|
| Justin|    29.5|          2|
+-------+--------+-----------+



* save dataframe

In [19]:
nameAvgAgeCount.rdd.saveAsTextFile("./nameAvgCount.txt")
nameAvgAgeCount.write.csv("./nameAvgCount.csv",header=True)

In [21]:
rdd_load=sc.textFile("nameAvgCount.txt")
print(rdd_load.collect())
df_reload = spark.read.load("./nameAvgCount.csv",
                           format="csv",
                           header=True,
                            inferSchema=True)
df_reload.show()

["Row(name='Michael', avg(age)=None, count(name)=1)", "Row(name='Andy', avg(age)=30.0, count(name)=1)", "Row(name='Justin', avg(age)=29.5, count(name)=2)"]
+-------+--------+-----------+
|   name|avg(age)|count(name)|
+-------+--------+-----------+
| Justin|    29.5|          2|
|Michael|    null|          1|
|   Andy|    30.0|          1|
+-------+--------+-----------+



* UDF: User Defined Funcitons

In [22]:
# define
spark.udf.register("length",lambda x:len(x))
# use
result = df_reload.selectExpr("length(name) as size")
result.show()

result = spark.sql("SELECT length(name) FROM people")
result.show()

+----+
|size|
+----+
|   6|
|   7|
|   4|
+----+

+------------+
|length(name)|
+------------+
|           6|
|           4|
|           7|
|           6|
+------------+

